In [4]:
import scipy
import numpy as np
from scipy.optimize import minimize
from sklearn import linear_model


def portfolio_optimizer(rho, lambda_, covariance_matrix, omega_p, beta_of_S_i, beta_target):
    def main_equation(omega):
        return -rho.dot(omega) + lambda_ * (omega - omega_p).dot(covariance_matrix.dot(omega - omega_p))

    def equation_1(omega):
        return np.dot(beta_of_S_i, omega) - beta_target

    def equation_2(omega):
        return np.sum(omega) - 1

    omega_0 = np.array([1 / 12] * 12)
    constraint = [{'type': 'eq', 'fun': equation_1}, {'type': 'eq', 'fun': equation_2}]
    boundary = scipy.optimize.Bounds(-2.0, 2.0, keep_feasible=True)

    return minimize(main_equation, omega_0, method='SLSQP', bounds=boundary, constraints=constraint, tol=1e-9).x


def omega_specific_period(return_return_period, factor_return_period,
                          return_variance_period, factor_variance_period, lambda_, beta_target, omega_p_):
    rf_return_period = np.asarray(factor_return_period['RF'])
    rM_rf_return_period = np.asarray(factor_return_period['Mkt-RF'])
    rSMB_return_period = np.asarray(factor_return_period['SMB'])
    rHML_return_period = np.asarray(factor_return_period['HML'])
    SPY_return_period = np.asarray(return_return_period['SPY'])
    
    return_etf_i = np.asarray(return_return_period)
    
    market_variance = np.var(SPY_return_period, ddof = 1)
    beta_of_etf = np.array([0.0] * 12)
    for i in range (12):
        cov_etf_mkt = np.cov(return_etf_i[:, i],SPY_return_period,ddof = 1)
        beta_of_etf[i] = cov_etf_mkt[0,1] / market_variance
    
    Fama_French_i = return_etf_i - rf_return_period.reshape(-1, 1)
    three_factors_as_row = np.array([rM_rf_return_period, rSMB_return_period, rHML_return_period])
    three_factors_as_column = three_factors_as_row.T
    
    ols = linear_model.LinearRegression().fit(three_factors_as_column, Fama_French_i)
    alpha = ols.intercept_
    ols_fama_b = ols.coef_

    # get rho of short period
    three_factor_target = three_factors_as_row[:, -1]
    rho_return_period = alpha + ols_fama_b.dot(three_factor_target) + rf_return_period[-1]
    
    # --------------------------------------------------------------------
    rf_variance_period = np.asarray(factor_variance_period['RF'])
    rM_rf_variance_period = np.asarray(factor_variance_period['Mkt-RF'])
    rSMB_variance_period = np.asarray(factor_variance_period['SMB'])
    rHML_variance_period = np.asarray(factor_variance_period['HML'])
    SPY_variance_period = np.asarray(return_variance_period['SPY'])
    
    return_etf_i_variance_period = np.asarray(return_variance_period)
    
    market_variance_variance_period = np.var(SPY_variance_period, ddof = 1)
    beta_etf_variance_period = np.array([0.0] * 12)
    for i in range (12):
        cov_im_variance_period = np.cov(return_etf_i_variance_period[:, i], SPY_variance_period, ddof = 1 )
        beta_etf_variance_period[i] = cov_im_variance_period[0, 1] / market_variance_variance_period
    
    Fama_French_i_variance_period = return_etf_i_variance_period - rf_variance_period.reshape(-1, 1)
    three_factors_row_variance_period = np.array([rM_rf_variance_period, rSMB_variance_period, rHML_variance_period])
    three_factors_column_var_period = three_factors_row_variance_period.T
    
    ols_variance_period = linear_model.LinearRegression().fit(three_factors_column_var_period,
                                                              Fama_French_i_variance_period)
    alpha_variance_period = ols_variance_period.intercept_
    ols_fama_b_var_period = ols_variance_period.coef_
    
    error_variance_period = Fama_French_i_variance_period.T \
                            - (alpha_variance_period.reshape(-1, 1)
                               + ols_fama_b_var_period.dot(three_factors_row_variance_period))
    error_cov = np.cov(error_variance_period, ddof = 1)
    error_cov_diagonal = np.diag(error_cov)
    error_diag_matrix = np.diag(error_cov_diagonal)

    cov_factors = np.cov(three_factors_row_variance_period, ddof = 1)
    cov_total = ols_fama_b_var_period.dot(cov_factors).dot(ols_fama_b_var_period.T) + error_diag_matrix


    
    result = portfolio_optimizer(rho_return_period, lambda_, cov_total, omega_p_, beta_etf_variance_period, beta_target)
    
    return result